In [1]:
import os
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense

# Define data file path
data_path = "hsg.json"

# # Define new data for today
# new_data = {
#     "o": [23450],  # Open price for today
#     "h": [23750],  # High price for today
#     "l": [23000],  # Low price for today
#     "c": [23000],  # Close price for today
#     "v": [14818600],  # Volume for today
# }

# Load data from JSON file
with open(data_path) as f:
    data = json.load(f)

# Extract features and target
X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
y = np.array(data["c"])

# Normalize data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshape data for GRU input (samples, timesteps, features)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Build the model
model = Sequential()
model.add(
    Bidirectional(
        GRU(units=256, return_sequences=True, input_shape=(X_scaled.shape[1], 1))
    )
)
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(units=256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(units=256)))
model.add(Dropout(0.2))
model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model using all data
model.fit(X_scaled, y_scaled, epochs=50, batch_size=2048, verbose=1)

# Predict tomorrow's value based on the last data point in the dataset
last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
predicted_value_scaled = model.predict(last_data_point)
predicted_value = scaler_y.inverse_transform(predicted_value_scaled)
print("Predicted value for tomorrow based on last data point:", predicted_value)

# # Preprocess new data for today and predict tomorrow's value
# new_X = np.column_stack(
#     (new_data["o"], new_data["h"], new_data["l"], new_data["c"], new_data["v"])
# )
# new_X_scaled = scaler_X.transform(new_X)
# new_X_scaled = new_X_scaled.reshape(1, new_X_scaled.shape[0], new_X_scaled.shape[1])

# predicted_value_scaled = model.predict(new_X_scaled)
# predicted_value = scaler_y.inverse_transform(predicted_value_scaled)
# print("Predicted value for tomorrow based on new data for today:", predicted_value)

2024-03-25 22:42:43.080875: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-25 22:42:43.080898: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-25 22:42:43.080904: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-25 22:42:43.081139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 22:42:43.081477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2024-03-25 22:42:46.508076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 6s 596ms/step - loss: 0.0870
Epoch 2/50
2/2 [==============================] - 0s 206ms/step - loss: 0.0595
Epoch 3/50
2/2 [==============================] - 1s 238ms/step - loss: 0.0156
Epoch 4/50
2/2 [==============================] - 0s 218ms/step - loss: 0.0271
Epoch 5/50
2/2 [==============================] - 0s 199ms/step - loss: 0.0058
Epoch 6/50
2/2 [==============================] - 0s 232ms/step - loss: 0.0143
Epoch 7/50
2/2 [==============================] - 0s 221ms/step - loss: 0.0130
Epoch 8/50
2/2 [==============================] - 0s 204ms/step - loss: 0.0043
Epoch 9/50
2/2 [==============================] - 1s 257ms/step - loss: 0.0036
Epoch 10/50
2/2 [==============================] - 0s 212ms/step - loss: 0.0062
Epoch 11/50
2/2 [==============================] - 0s 197ms/step - loss: 0.0014
Epoch 12/50
2/2 [==============================] - 0s 196ms/step - loss: 0.0023
Epoch 13/50
2/2 [==============================] - 0s 191ms/